# differentiating a QP wrt a single variable

consider the QP

| minimize | $$ \frac{1}{2} x^T Q x + q^T x$$ |
| -------- | -------------------------------- |
| s.t.     | $$ G x \leq h, x \in R^2, h \in R$$ |

where `Q`, `q`, `G` are fixed and `h` is the single parameter.

here i've tried to differentiate the QP wrt `h` and compared the Julia result (can be solved by hand even!) with
- https://github.com/cvxgrp/cvxpylayers#tensorflow-2
- eqn (6) of https://arxiv.org/pdf/1703.00443.pdf 

Assuming 
```
Q = [[4, 1], [1, 2]]
q = [1, 1]
G = [1, 1]
```
and begining with a starting value of `h=-1`

few values just for reference

| variable | optimal value | note |
| -- | ---- | --- |
| x* | [-0.25; -0.75] | primal optimal | 
| 𝜆∗ | -0.75 | dual optimal | 


## 1) solving it in cvxlayers

In [ ]:
import cvxpy as cp
import tensorflow as tf
from cvxpylayers.tensorflow import CvxpyLayer

n, m = 2, 1
x = cp.Variable(n)
Q = np.array([[4, 1], [1, 2]])
q = np.array([1, 1])
G = np.array([1, 1])
h = cp.Parameter(m)
constraints = [G@x <= h]
objective = cp.Minimize(0.5*cp.quad_form(x, Q) + q.T @ x)
problem = cp.Problem(objective, constraints)
assert problem.is_dpp()

cvxpylayer = CvxpyLayer(problem, parameters=[h], variables=[x])
h_tf = tf.Variable([-1.0])  # set a starting value

with tf.GradientTape() as tape:
  # solve the problem, setting the values of h to h_tf
  solution, = cvxpylayer(h_tf)

  summed_solution = tf.math.reduce_sum(solution)
  
# note - solution is [-0.25, -0.75]
#        summed_solution is (-0.25) + (-0.75)

# cvxpylayers allows gradient of the summed solution only, with respect to h
gradh = tape.gradient(summed_solution, [h_tf])

## 2) deriving result using matrix inversion
refer eqn (6) of https://arxiv.org/pdf/1703.00443.pdf

if we assume `h` as the only parameter and `Q`,`q`,`G` as fixed problem data - also note that our QP doesn't involves `Ax=b` constraint - then eqn (6) reduces to 
$$ 
\begin{gather}
 \begin{bmatrix} 
     Q & g^T \\
     \lambda^* g & g z^* - h
 \end{bmatrix}
 \begin{bmatrix} 
     dz \\
     d \lambda
 \end{bmatrix}
 =
  \begin{bmatrix}
   0 \\
   \lambda^* dh
   \end{bmatrix}
\end{gather}
$$

now to find the jacobians $$ \frac{\partial z}{\partial h}, \frac{\partial \lambda}{\partial h}$$
we substitute `dh = I = [1]` (and plug in other values) to get
$$ 
\begin{gather}
 \begin{bmatrix} 
     4 & 1 & 1 \\
     1 & 2 & 1 \\
     -0.75 & -0.75 & 0
 \end{bmatrix}
 \begin{bmatrix} 
     \frac{\partial z_1}{\partial h} \\
     \frac{\partial z_2}{\partial h} \\
     \frac{\partial \lambda}{\partial h}
 \end{bmatrix}
 =
  \begin{bmatrix}
   0 \\
   0 \\
   -0.75
   \end{bmatrix}
\end{gather}
$$

the solution is
$$ \frac{\partial z_1}{\partial h} = 0.25, \frac{\partial z_2}{\partial h} = 0.75, \frac{\partial \lambda}{\partial h} = -1.75 $$

## 3) solving using MOI, Dualization.jl, LinearAlgebra

In [21]:
using Random
using MathOptInterface
using Dualization
using OSQP
using LinearAlgebra

const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities;

n = 2 # variable dimension
m = 1; # no of inequality constraints

Q = [4. 1.;1. 2.]
q = [1.; 1.]
G = [1. 1.;]
h = [-1.;]   # initial values set


# create the optimizer
model = MOI.instantiate(OSQP.Optimizer, with_bridge_type=Float64)
x = MOI.add_variables(model, n);

In [25]:
# define objective

quad_terms = MOI.ScalarQuadraticTerm{Float64}[]
for i in 1:n
    for j in i:n # indexes (i,j), (j,i) will be mirrored. specify only one kind
        push!(quad_terms, MOI.ScalarQuadraticTerm(Q[i,j],x[i],x[j]))
    end
end

objective_function = MOI.ScalarQuadraticFunction(MOI.ScalarAffineTerm.(q, x),quad_terms,0.)
MOI.set(model, MOI.ObjectiveFunction{MOI.ScalarQuadraticFunction{Float64}}(), objective_function)
MOI.set(model, MOI.ObjectiveSense(), MOI.MIN_SENSE)

In [26]:
# add constraint
MOI.add_constraint(
    model,
    MOI.ScalarAffineFunction(MOI.ScalarAffineTerm.(G[1,:], x), 0.),
    MOI.LessThan(h[1])
)

In [27]:
MOI.optimize!(model)

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 2, constraints m = 1
          nnz(P) + nnz(A) = 5
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -2.3138e-01   1.79e-01   4.00e-01   1.00e-01   4.89e-05s
  50  -1.2500e-01   3.05e-13   5.59e-12   7.14e+00   8.11e-05s

status:               solved
number of iterations:

In [28]:
@assert MOI.get(model, MOI.TerminationStatus()) in [MOI.LOCALLY_SOLVED, MOI.OPTIMAL]

In [29]:
x̄ = MOI.get(model, MOI.VariablePrimal(), x)

2-element Array{Float64,1}:
 -0.2499999999999395
 -0.7499999999997555

## obtaining lambda*

In [31]:
joint_object    = dualize(model)
dual_model_like = joint_object.dual_model # this is MOI.ModelLike, not an MOI.AbstractOptimizer; can't call optimizer on it
primal_dual_map = joint_object.primal_dual_map;

In [32]:
# copy the dual model objective, constraints, and variables to an optimizer
dual_model = MOI.instantiate(OSQP.Optimizer, with_bridge_type=Float64)
MOI.copy_to(dual_model, dual_model_like)

# solve dual
MOI.optimize!(dual_model);

-----------------------------------------------------------------
           OSQP v0.6.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 3, constraints m = 3
          nnz(P) + nnz(A) = 10
settings: linear system solver = qdldl,
          eps_abs = 1.0e-03, eps_rel = 1.0e-03,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 4000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: off, time_limit: off

iter   objective    pri res    dua res    rho        time
   1  -7.8022e-01   1.02e+00   1.50e+02   1.00e-01   5.35e-05s
  25   1.2474e-01   2.70e-04   7.11e-04   1.00e-01   7.56e-05s

status:               solved
number of iterations

In [34]:
map = primal_dual_map.primal_con_dual_var

for con_index in keys(map)
    λ = MOI.get(dual_model, MOI.VariablePrimal(), map[con_index][1])
    println(λ)
end

-0.7500438286088407


In [36]:
LHS = [4 1 1; 1 2 1; 1 1 0]  # of eqn (6)

3×3 Array{Int64,2}:
 4  1  1
 1  2  1
 1  1  0

In [37]:
RHS = [0; 0; 1]  # of eqn (6)

3-element Array{Int64,1}:
 0
 0
 1

In [38]:
pp \ qq  # voila

3-element Array{Float64,1}:
  0.25
  0.75
 -1.75